### 환경 install

In [1]:
#pip install langchain-google-genai


In [2]:
#pip install langchain langchain-community langchain-core langchain-text-splitters langchain-google-genai faiss-cpu pymupdf

In [3]:
#pip install langchain-postgres psycopg2-binary

In [4]:
#pip install google-generativeai

In [5]:
# pip install langchain langchain-community langchain-openai

In [6]:
# pip install langchain-openai

In [7]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_community.document_loaders import PyMuPDFLoader
# from langchain_community.vectorstores import FAISS
# from langchain_core.output_parsers import StrOutputParser
# from langchain_core.runnables import RunnablePassthrough
# from langchain_core.prompts import PromptTemplate
# from langchain_community.document_loaders import TextLoader
# from langchain_community.vectorstores import PGVector
# # from langchain_community.retrievers import MergerRetriever  

# # 변경 부분 - Google genai, and PGVector
# from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
# from langchain_postgres.vectorstores import PGVector
# from langchain_community.document_loaders import DirectoryLoader, TextLoader
# import time

import os
import pandas as pd
import numpy as np
import openai
from datetime import datetime
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import PGVector
from dotenv import load_dotenv
# from langchain_community.retrievers import MergerRetriever  
from langchain_openai import ChatOpenAI

# 변경 부분 - Google genai, and PGVector
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_postgres.vectorstores import PGVector
from langchain_community.document_loaders import DirectoryLoader, TextLoader
import time
from dotenv import load_dotenv


#Openai 추가
from langchain_openai import OpenAIEmbeddings
from langchain_postgres import PGVector

In [9]:
# API KEY 체크
load_dotenv(override=True)

# API 키 확인
api_key = os.getenv('OPENAI_API_KEY')
print(f"API 키: {api_key[:8]}")
CONNECTION_STRING = os.getenv("DATABASE_URL")
print(f"DB connection string : {CONNECTION_STRING[:3]}")

# 만약 None이면 문제!
if api_key is None:
    print("❌ .env 파일에 OPENAI_API_KEY가 없습니다!")

if CONNECTION_STRING is None:
    print("❌ .env 파일에 CONNECTION_STRING가 없습니다!")

API 키: sk-proj-


## 임베딩 Ai 연결, DB 연결

In [10]:
# 단계 3: 임베딩(Embedding) 생성
#openai 사용 
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
# 단계 4: DB 생성(Create DB) 및 저장
# 벡터스토어를 생성합니다.


# 임베딩 추가 없이 DB만 호출하고 싶을때는 아래 코드 활성화  
vectorstore = PGVector(
    embeddings=embeddings,
    collection_name="SI_try",
    connection=CONNECTION_STRING,  # connection_string이 아니라 connection
)


In [11]:
# # langchain 소스코드 확인하기
# # 흥미로운 내용 많네요 
# ??PGVector


# 일반 유사도 검색(기존)

In [12]:
# 코사인 거리(cosine distance) = 1 - cos(θ)
# - 범위: 0 ~ 2 (보통 0 ~ 1)
# - 0에 가까울수록 유사

#k는 문서 개수, 정의 안하면 기본 4개 
results = vectorstore.similarity_search_with_score("투썸플레이스 영수증프린터", k=10)
for i, (doc, score) in enumerate(results, 1):
    print(f"{i}위 - 유사도: {score:.4f}")
    print(f"내용: {doc.page_content[:100]}...")
    print("---")

1위 - 유사도: 0.4128
내용: [접수번호 26440]
접수일시: 2025-05-23 20:21
가맹점: 투썸플레이스 - 투썸플레이스(당진시청점)

문제 요약: 영수증프린터 출력 안됨

상세 내용:
영수증 출력 ...
---
2위 - 유사도: 0.4323
내용: [접수번호 15386]
접수일시: 2025-03-24 10:03
가맹점: 투썸플레이스 - 투썸(인천검단금호점)

문제 요약: 영수증 출력 관련

상세 내용:
영수증 프린터 출력안됨...
---
3위 - 유사도: 0.4364
내용: [접수번호 7572]
접수일시: 2025-02-11 19:58
가맹점: 투썸플레이스 - 투썸플레이스(청주비하점)

문제 요약: 영수증 출력 안 됨

상세 내용:
영수증 출력 안 됨...
---
4위 - 유사도: 0.4400
내용: [접수번호 14579]
접수일시: 2025-03-19 11:44
가맹점: 투썸플레이스 - 투썸플레이스(대구지산광장점)

문제 요약: POS프린터 출력이상(흐림, 잘림, 안나옴)

...
---
5위 - 유사도: 0.4406
내용: [접수번호 25969]
접수일시: 2025-05-21 12:18
가맹점: 투썸플레이스 - 투썸 구리교문사거리점

문제 요약: 영수증 프린터 관련 문의

상세 내용:
1. 장비 : ...
---
6위 - 유사도: 0.4430
내용: [접수번호 39310]
접수일시: 2025-08-03 11:44
가맹점: 투썸플레이스 - 투썸플레이스(평택석정점)

문제 요약: POS프린터 출력이상(흐림, 잘림, 안나옴)

상세...
---
7위 - 유사도: 0.4459
내용: [접수번호 67808]
접수일시: 2025-12-22 16:08
가맹점: 투썸플레이스 - 투썸플레이스(서동탄역점)

문제 요약: POS프린터 출력이상(흐림, 잘림, 안나옴)

상세...
---
8위 - 유사도: 0.4481
내용: [접수번호 42562]
접수일시: 2025-08-22 09:45
가맹점: 투썸플레이스 - 투썸플레이스(구리역점)

문제 요약: POS프린

# MMR 사용 

In [13]:
#MMR 사용, lambda_mult = 1으로 설정하여 유사도 검색과 같이 만들기 

results = vectorstore.max_marginal_relevance_search_with_score("투썸플레이스 영수증프린터", k=3, lambda_mult=1)

for i, (doc, score) in enumerate(results, 1):
    print(f"{i}위 - 유사도: {score:.4f}")
    print(f"내용: {doc.page_content[:100]}...")
    print("---")

# print(len(results))
# print(results[0].page_content)

1위 - 유사도: 0.4128
내용: [접수번호 26440]
접수일시: 2025-05-23 20:21
가맹점: 투썸플레이스 - 투썸플레이스(당진시청점)

문제 요약: 영수증프린터 출력 안됨

상세 내용:
영수증 출력 ...
---
2위 - 유사도: 0.4323
내용: [접수번호 15386]
접수일시: 2025-03-24 10:03
가맹점: 투썸플레이스 - 투썸(인천검단금호점)

문제 요약: 영수증 출력 관련

상세 내용:
영수증 프린터 출력안됨...
---
3위 - 유사도: 0.4364
내용: [접수번호 7572]
접수일시: 2025-02-11 19:58
가맹점: 투썸플레이스 - 투썸플레이스(청주비하점)

문제 요약: 영수증 출력 안 됨

상세 내용:
영수증 출력 안 됨...
---


In [14]:
#MMR 사용, lambda_mult = 0으로 설정하여 다양성 위주로 검색 

results = vectorstore.max_marginal_relevance_search_with_score("투썸플레이스 영수증프린터", k=3, lambda_mult=0)

for i, (doc, score) in enumerate(results, 1):
    print(f"{i}위 - 유사도: {score:.4f}")
    print(f"내용: {doc.page_content[:100]}...")
    print("---")

# print(len(results))
# print(results[0].page_content)

1위 - 유사도: 0.4128
내용: [접수번호 26440]
접수일시: 2025-05-23 20:21
가맹점: 투썸플레이스 - 투썸플레이스(당진시청점)

문제 요약: 영수증프린터 출력 안됨

상세 내용:
영수증 출력 ...
---
2위 - 유사도: 0.4632
내용: [접수번호 53284]
접수일시: 2025-10-13 09:35
가맹점: 투썸플레이스 - 투썸(동탄역앨리스빌점)

문제 요약: POS프린터 출력이상(흐림, 잘림, 안나옴)

상세 ...
---
3위 - 유사도: 0.4643
내용: [접수번호 18558]
접수일시: 2025-04-09 15:56
가맹점: 투썸플레이스 - 투썸플레이스(인천옥련점)

문제 요약: 키오스크 관련

상세 내용:
키오스크 영수증프린터 ...
---


## Retriever에서 검색 방법 바꾸는 법

In [15]:
# 단계 5: 검색기(Retriever) 생성
# 문서에 포함되어 있는 정보를 검색하고 생성합니다.
# retriever = vectorstore.as_retriever()

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 30}
)
# 기본값: search_type="similarity"
print(retriever.search_type)  


# # mmr로 바꾸고 싶을 때
# retriever = vectorstore.as_retriever(search_type="mmr")
# print(retriever.search_type)  


similarity


In [16]:
# 검색기에 쿼리를 날려 검색된 chunk 결과를 확인합니다.
retriever.invoke("'투썸', '키오스크','부산덕천젊음의거리' 해결 사례?")

[Document(id='1744f474-0693-40f2-be6f-94d666988346', metadata={'no': 2004, 'date': '2025-01-11', 'type': '원격', 'year': 2025, 'month': 1, 'summary': '키오스크 관련', 'category': '투썸플레이스', 'store_name': '투썸플레이스(수원금곡점)', 'sub_category': '키오스크', 'main_category': '장비'}, page_content='[접수번호 2004]\n접수일시: 2025-01-11 16:07\n가맹점: 투썸플레이스 - 투썸플레이스(수원금곡점)\n\n문제 요약: 키오스크 관련\n\n상세 내용:\n키오스크 해결되었다 하심\n\n처리 결과:\n원격처리\n\n분류: 장비 > 키오스크\n처리 방식: 원격\n담당자: 고정현'),
 Document(id='f283ce0a-e03f-4b80-8c86-f6c3f5266134', metadata={'no': 51541, 'date': '2025-10-02', 'type': '원격', 'year': 2025, 'month': 10, 'summary': '이전설치 요청', 'category': '투썸플레이스', 'store_name': '투썸 부산덕천젊음의거리점', 'sub_category': '서비스요청', 'main_category': '장비'}, page_content='[접수번호 51541]\n접수일시: 2025-10-02 11:33\n가맹점: 투썸플레이스 - 투썸 부산덕천젊음의거리점\n\n문제 요약: 이전설치 요청\n\n상세 내용:\n키오스크 2대 위치변경 문의\n\n처리 결과:\n무선 사용매장으로 위치이동 후 전원만 연결하면 되는점 안내\n\n분류: 장비 > 서비스요청\n처리 방식: 원격\n담당자: 정일웅'),
 Document(id='be84e13d-f15d-4028-9a40-7c2225b83404', metadata={'no': 13174, 'date': '20

In [17]:
# 단계 6: 프롬프트 생성(Create Prompt)
# 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(
    """You are an assistant for AS call center worker who needs information for error case - answer tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Provide several cases including details and then one summary sentence. 
Answer in Korean.

IMPORTANT: Ignore documents without exact match, Priority is user's query. Strictly follow the query.


#Context: 
{context}

#Question:
{question}

#Answer:"""
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
from langchain_core.prompts import ChatPromptTemplate

In [19]:
# answer_prompt = ChatPromptTemplate.from_messages([
#     ("system", """You are a customer service case analysis expert. 
# Based on the retrieved consultation cases, provide your answer in the following format:

# 1. Present **5 cases** that match the query in the following format:
#    - Store Name: 
#    - Franchise Category:
#    - Receipt Date:
#    - Issue Summary:
#    - Detailed Content:
#    - Resolution Result:

# 2. At the end, provide a brief **overall analysis** of all cases

# **IMPORTANT: Respond in Korean, even though these instructions are in English.**
# """),
#     ("user", """Question: {question}

# Search Results:
# {context}

# Please answer according to the format above, in Korean.""")
# ])

# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | answer_prompt
#     | llm
#     | StrOutputParser()
# )

In [20]:
# 단계 8: 체인(Chain) 생성
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# chain = (
#     {"context": retriever, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

In [21]:
%%time
question = "투썸 매장에서 POS 프린터 오류 어떻게 처리했어?"
response = chain.invoke(question)
print(f"💬 답변:\n{response}")

💬 답변:
투썸 매장에서 POS 프린터 오류 처리 사례는 다음과 같습니다:

1. **투썸플레이스(강동역점)**  
   - **접수일시**: 2025-06-26  
   - **문제 요약**: 영수증 출력이 간헐적으로 안됨  
   - **처리 결과**: 매장에서 증상을 정확히 알고 있는 직원이 접수 안내를 하기로 함.  

2. **투썸 대림삼거리(4)**  
   - **접수일시**: 2025-09-08  
   - **문제 요약**: 출력 설정 문의  
   - **처리 결과**: 픽업프린터 사용으로 설정 후 매장에서 메뉴프린터 연결 설정 방법 안내.  

3. **투썸(북대구IC주유소DT점)**  
   - **접수일시**: 2025-05-02  
   - **문제 요약**: 2번 프린터 출력 안됨  
   - **처리 결과**: 출장 점검 요청, 프린터 및 USB 선 재연결, 드라이버 재설치 후에도 동일 증상 발생.  

4. **투썸플레이스(송산그린시티점)**  
   - **접수일시**: 2025-05-25  
   - **문제 요약**: 주문서 출력 안됨  
   - **처리 결과**: 매장 공유기 바뀌어 IP 세팅 후 정상 작동.  

5. **투썸(대전용문점)**  
   - **접수일시**: 2025-06-16  
   - **문제 요약**: 영수증 프린터 전원 안켜짐  
   - **처리 결과**: 전원선 및 어댑터 재연결 후에도 동일 증상 발생, 방문 점검 요청.  

이와 같이, 투썸 매장에서는 POS 프린터 오류 발생 시 원격 지원이나 출장 점검을 통해 문제를 해결하고 있습니다.
CPU times: total: 78.1 ms
Wall time: 28.7 s


In [22]:
%%time
question = "1월에서 2월 사이 투썸플레이스 키오스크 문의사항을 보여줘."
response = chain.invoke(question)
print(f"💬 답변:\n{response}")

💬 답변:
다음은 2025년 1월에서 2월 사이에 접수된 투썸플레이스 키오스크 관련 문의사항입니다.

1. **접수번호 5394**
   - **접수일시:** 2025-01-31 11:19
   - **가맹점:** 투썸플레이스 - 투썸(부산경성대역)
   - **문제 요약:** 행사 프로모션 진행하는데 익일 반영 여부 문의
   - **상세 내용:** 행사 안뜨게 프로그램상 설정 가능한지 문의
   - **처리 결과:** 포스트 설정이라 당사에서 프로그램상 프로모션이 안뜨게 설정은 어려운 점 안내.

2. **접수번호 2982**
   - **접수일시:** 2025-01-16 16:46
   - **가맹점:** 투썸플레이스 - 투썸(2001아울렛중계점)
   - **문제 요약:** 2번 키오스크 DID 연동 설정 안되어있어 문의
   - **상세 내용:** 포스트 DID 사용여부 사용으로 변경하고 사용 안내.
   - **처리 결과:** 포스트 DID 사용여부 사용으로 변경하고 사용 안내.

3. **접수번호 7520**
   - **접수일시:** 2025-02-11 16:14
   - **가맹점:** 투썸플레이스 - 투썸플레이스(염창역점)
   - **문제 요약:** 키오스크 설치 일정 문의
   - **상세 내용:** 키오스크 배출기 등 주변장치 도착 시 연락 안내.
   - **처리 결과:** 키오스크 배출기 등 주변장치 도착 시 연락주시라 안내.

4. **접수번호 11013**
   - **접수일시:** 2025-02-28 10:39
   - **가맹점:** 투썸플레이스 - 투썸_칠곡휴게소점(상행)
   - **문제 요약:** 키오스크 관련
   - **상세 내용:** 키오스크 2대 모두 카드결제 시 타임아웃 문구 뜸.
   - **처리 결과:** 원격으로 카드결제 시 모두 정상. 사용 안내.

요약하자면, 1월과 2월 사이에 투썸플레이스 키오스크에 대한 여러 문의가 있었으며, 주로 행사 프로모션 설정, DID 연동, 설치 일정, 카드결제 관련 문제들

"You are an assistant for AS call center worker who needs information for error case - answer tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nProvide several cases including details and then one summary sentence. \nAnswer in Korean.\n\nIMPORTANT: Ignore documents without exact match, Priority is user's query. Strictly follow the query.\n\n\n#Context: \n{context}\n\n#Question:\n{question}\n\n#Answer:"

In [40]:
import tiktoken




# 1. 인코더 초기화 (GPT-4o-mini는 cl100k_base 사용)
encoding = tiktoken.encoding_for_model("gpt-4o-mini")

# 2. 각 부분의 토큰 수 계산
def count_tokens(text):
    return len(encoding.encode(text))

# 3. 실제 측정
question = "1월에서 2월 사이 투썸플레이스 키오스크 문의사항을 보여줘."

# Context 가져오기
retrieved_docs = retriever.invoke(question)
context = "\n".join([doc.page_content for doc in retrieved_docs])

# # 토큰 수 계산
# prompt_tokens = count_tokens(prompt.template)  # 템플릿 고정 부분
# context_tokens = count_tokens(context)
# question_tokens = count_tokens(question)
# output_tokens = count_tokens(response)

# total_input_tokens = prompt_tokens + context_tokens + question_tokens
# total_output_tokens = output_tokens

# total_token_price = total_input_tokens * input_price + output_tokens * output_price

# print(f"📊 토큰 사용량:")
# print(f"  입력 토큰: {total_input_tokens:,}")
# print(f"    - Prompt: {prompt_tokens}")
# print(f"    - Context: {context_tokens}")
# print(f"    - Question: {question_tokens}")
# print(f"  출력 토큰: {total_output_tokens:,}")
# print(f"  총 토큰: {total_input_tokens + total_output_tokens:,}")

In [41]:
# 토큰 수, 가격 계산
# 모델 마다 토큰 당 가격이 다름 : gpt-4o-mini의 경우 1M token 당 input $0.15	캐시 input $0.075	output$0.60
# gpt-4o-mini price
input_price = 0.15/1000000
output_price = 0.6/1000000

# 환율
d2won = 1458

prompt_tokens = count_tokens(prompt.template)  # 템플릿 고정 부분
context_tokens = count_tokens(context)
question_tokens = count_tokens(question)
output_tokens = count_tokens(response)

total_input_tokens = prompt_tokens + context_tokens + question_tokens
total_output_tokens = output_tokens

total_token_price = total_input_tokens * input_price + output_tokens * output_price
Kwon_total_token_price = d2won * total_token_price

print(f"📊 토큰 사용량:")
print(f"  입력 토큰: {total_input_tokens:,}")
print(f"    - Prompt: {prompt_tokens}")
print(f"    - Context: {context_tokens}")
print(f"    - Question: {question_tokens}")
print(f"  출력 토큰: {total_output_tokens:,}")
print(f"  총 토큰: {total_input_tokens + total_output_tokens:,}")
# 유효숫자 2개
print(f"총 가격 ($) : {total_token_price:.2g}")
print(f"총 가격 (원) : {Kwon_total_token_price:.2g}")


📊 토큰 사용량:
  입력 토큰: 4,584
    - Prompt: 99
    - Context: 4463
    - Question: 22
  출력 토큰: 576
  총 토큰: 5,160
총 가격 ($) : 0.001
총 가격 (원) : 1.5


In [42]:
#한달에 드는 총 비용 
# 주어진 데이터
monthly_mean = 5840.5  # 한 달 평균 건수
monthly_std = 651.9    # 표준편차
questions_per_case = 1  # 1건당 질문 수
cost_per_question = Kwon_total_token_price  # 1 질문당 비용 (원)

# 계산
total_questions_mean = monthly_mean * questions_per_case
total_questions_std = monthly_std * questions_per_case

total_cost_mean = total_questions_mean * cost_per_question
total_cost_std = total_questions_std * cost_per_question

print("=" * 60)
print("💰 한 달 LLM 비용 계산")
print("=" * 60)
print(f"월 평균 케이스:        {monthly_mean:,.1f}건")
print(f"1건당 질문 수:         {questions_per_case}번")
print(f"1질문당 비용:          {cost_per_question}원")
print()
print(f"월 총 질문 수:         {total_questions_mean:,.1f}회")
print(f"                       (±{total_questions_std:,.1f}회)")
print()
print(f"💵 월 평균 비용:       {total_cost_mean:,.0f}원")
print(f"   (±{total_cost_std:,.0f}원)")
print()
print(f"예상 범위:")
print(f"  최소: {total_cost_mean - total_cost_std:,.0f}원")
print(f"  평균: {total_cost_mean:,.0f}원")
print(f"  최대: {total_cost_mean + total_cost_std:,.0f}원")
print()
print(f"💳 연간 예상 비용:     {total_cost_mean * 12:,.0f}원")
print(f"                       ({total_cost_mean * 12 / 10000:.1f}만원)")

💰 한 달 LLM 비용 계산
월 평균 케이스:        5,840.5건
1건당 질문 수:         1번
1질문당 비용:          1.5064056원

월 총 질문 수:         5,840.5회
                       (±651.9회)

💵 월 평균 비용:       8,798원
   (±982원)

예상 범위:
  최소: 7,816원
  평균: 8,798원
  최대: 9,780원

💳 연간 예상 비용:     105,578원
                       (10.6만원)
